In [5]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import sys
import os
from sklearn.model_selection import StratifiedShuffleSplit
import shutil

In [6]:
def DeleteAllFiles(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
        return 'Remove All Files'
    else:
        return 'Directory Not Found'

In [7]:
#obj
xml_dir = "D:/data/한전data/data_split/train/labels/"
img_dir = "D:/data/한전data/data_split/train/images/"

xml_list = os.listdir(xml_dir)
img_list = os.listdir(img_dir)

In [14]:
# 80, 81f로 class 변경
class_index = []
img_index = []
count = -1
cls_0, cls_1, cls_2, cls_3, cls_4, cls_5, cls_6, cls_7 = 0, 0, 0, 0, 0, 0, 0, 0
nagari = 0

for c in xml_list:
    with open(xml_dir + c, "r", encoding='utf-8') as f:
        lines = f.readlines()
        count += 1
        for line in lines:
            line_s = line.split()
#             print(line_s[0])
            if line_s[0] == '0':
                cls_0 += 1
            elif line_s[0] == '1':
                cls_1 += 1
            elif line_s[0] == '2':
                cls_2 += 1
            elif line_s[0] == '3':
                cls_3 += 1
            elif line_s[0] == '4':
                cls_4 += 1
            elif line_s[0] == '5':
                cls_5 += 1
            elif line_s[0] == '6':
                cls_6 += 1
            elif line_s[0] == '7':
                cls_7 += 1
            else:
                no_class += 1
        
            class_index.append(line_s[0])
            img_index.append(count)

print(cls_0, cls_1, cls_2, cls_3, cls_4, cls_5, cls_6, cls_7)

class_index = list(map(int, class_index))

class_index = np.array(class_index)
img_index = np.array(img_index)

2465 2757 215 2648 8560 1930 5010 1804


In [15]:
class_num = [cls_0, cls_1, cls_2, cls_3, cls_4, cls_5, cls_6, cls_7]

class_num_list = list(map(int, (class_num[i] * 0.8 for i in range(len(class_num)))))

In [16]:
class_num

[2465, 2757, 215, 2648, 8560, 1930, 5010, 1804]

In [10]:
# df_img_index = pd.DataFrame(img_index)
# df_img_index.to_csv("")

# df_class_index = pd.DataFrame(class_index)
# df_class_index.to_csv("")

In [17]:
#class의 비율을 맞춰서 train, test data 나누기

#5번 분할
splits = 5
seed = 0
np.random.seed(seed)

shufflesplit = StratifiedShuffleSplit(n_splits=splits, random_state=seed, test_size=0.2)

train_random = []
test_random = []
train_img_final = []

limit_arr = class_num_list
raw_num = class_num

xml_arr = np.copy(xml_list)

print(shufflesplit)
print("="*100)

for train_index, test_index in shufflesplit.split(img_index, class_index):
    train_random.append(train_index)
    test_random.append(test_index)
    print("train Index:", train_index)
    print("test index:", test_index)
    print("-"*100)
    
    num_arr = list(range(3383))
    train_idx = []
    test_idx = []
    
    cnt_0_all, cnt_1_all, cnt_2_all, cnt_3_all, cnt_4_all, cnt_5_all, cnt_6_all, cnt_7_all = 0, 0, 0, 0, 0, 0, 0, 0

    for i in range(len(train_index)):
        num_idx = img_index[train_index][i]
        cls_sum_idx = class_index[np.where(img_index == num_idx)]
        for j in range(0,8):
            globals()['cnt_'+str(j)] = (cls_sum_idx == j).sum()
        if num_idx in num_arr:
            cnt_0_all += cnt_0
            cnt_1_all += cnt_1
            cnt_2_all += cnt_2
            cnt_3_all += cnt_3
            cnt_4_all += cnt_4
            cnt_5_all += cnt_5
            cnt_6_all += cnt_6
            cnt_7_all += cnt_7
            num_arr.remove(num_idx)
    #         qwe.append(num_idx)
            for k in range(0,8):
                globals()['bool_'+str(k)] = (globals()['cnt_'+str(k)+'_all'] < limit_arr[k])
            if not all([bool_0, bool_1, bool_2, bool_3, bool_4, bool_5, bool_6, bool_7]):
                false_conds = [n for n, cond in enumerate([bool_0, bool_1, bool_2, bool_3, bool_4, bool_5, bool_6, bool_7]) if not cond]
                zero_idx = []
                for m in false_conds:
                    globals()['bool_zero_'+str(m)] = (globals()['cnt_'+str(m)] == 0)
                    zero_idx.append(globals()['bool_zero_'+str(m)])
                if all(zero_idx):
                    train_idx.append(num_idx)
                else:
                    test_idx.append(num_idx)
            else:
                train_idx.append(num_idx)
  
    print(len(train_idx))
    train_img_final.append(np.sort(train_idx))
    cls_0, cls_1, cls_2, cls_3, cls_4, cls_5, cls_6, cls_7 = 0, 0, 0, 0, 0, 0, 0, 0
    xml_train_arr = xml_arr[train_idx]

    for c in xml_train_arr:
        with open(xml_dir + c, "r", encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                line_s = line.split()
                cls_num = int(line_s[0])
                if cls_num == 0:
    #                 print("asd")
                    cls_0 += 1
                elif cls_num == 1:
                    cls_1 += 1
                elif cls_num == 2:
                    cls_2 += 1
                elif cls_num == 3:
                    cls_3 += 1
                elif cls_num == 4:
                    cls_4 += 1
                elif cls_num == 5:
                    cls_5 += 1
                elif cls_num == 6:
                    cls_6 += 1
                elif cls_num == 7:
                    cls_7 += 1

    print(cls_0, cls_1, cls_2, cls_3, cls_4, cls_5, cls_6, cls_7)
    result = []
    for all_num in range(len(raw_num)):
        result.append(raw_num[all_num] - globals()['cls_'+str(all_num)])
        print(globals()['cls_'+str(all_num)]/raw_num[all_num])

    print("*"*100)

StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.2,
            train_size=None)
train Index: [11368  9177  5168 ...  7857 19538 21698]
test index: [ 6054  4646   333 ... 24061 10706 24445]
----------------------------------------------------------------------------------------------------
1946
1971 2171 157 2094 6650 1432 3601 1360
0.7995943204868154
0.7874501269495828
0.7302325581395349
0.790785498489426
0.7768691588785047
0.7419689119170985
0.7187624750499002
0.753880266075388
****************************************************************************************************
train Index: [16013 18442  7576 ... 16525 13317  3151]
test index: [12396 12311 13796 ... 15480 11164 16592]
----------------------------------------------------------------------------------------------------
1884
1968 2141 147 2101 6456 1474 3524 1345
0.7983772819472617
0.7765687341313021
0.6837209302325581
0.793429003021148
0.7542056074766356
0.7637305699481866
0.7033932135728543
0.745565410199

In [22]:
#분할된 data간의 유사도
for i in range(0,5):
    main = train_img_final[i]
    globals()["train_final_bin_" + str(i)] = []
    for j in range(len(xml_list)):
        if j in main:
            globals()["train_final_bin_" + str(i)].append(1)
        else:
            globals()["train_final_bin_" + str(i)].append(0)

In [24]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [25]:
doc0 = train_final_bin_0
doc1 = train_final_bin_1
doc2 = train_final_bin_2
doc3 = train_final_bin_3
doc4 = train_final_bin_4

# print(len(doc0), len(doc1), len(doc2), len(doc3), len(doc4))
# print((doc0), (doc1), (doc2), (doc3), (doc4))


for i in range(0,5):
    for j in range(0,5):
        if i == 0:
            if i != j:
                print(f'문서 {i}와 문서{j}의 유사도 :',np.round(cos_sim(globals()['doc'+str(i)], globals()['doc'+str(j)]), 3))
        elif i != 0:
            if i < j:
                print(f'문서 {i}와 문서{j}의 유사도 :',np.round(cos_sim(globals()['doc'+str(i)], globals()['doc'+str(j)]), 3))
        


문서 0와 문서1의 유사도 : 0.694
문서 0와 문서2의 유사도 : 0.705
문서 0와 문서3의 유사도 : 0.691
문서 0와 문서4의 유사도 : 0.696
문서 1와 문서2의 유사도 : 0.686
문서 1와 문서3의 유사도 : 0.7
문서 1와 문서4의 유사도 : 0.683
문서 2와 문서3의 유사도 : 0.706
문서 2와 문서4의 유사도 : 0.696
문서 3와 문서4의 유사도 : 0.696
